### geemap.js_snippet_to_py(js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True)

In [ ]:
import geemap
import ee

In [ ]:
Map = geemap.Map()
Map

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}
Map.setCenter(-122.1899, 37.5010, 10)      # 按地图视图按经纬度坐标来
Map.addLayer(image, vizParams, 'false color composite')
# bands参数需要一个有且只含三个元素的列表，列表里每一个参数为影像的一个波段的名称
# 用这三个波段来映射到RGB的红绿蓝三种颜色中
# min,max设置显示波段值的颜色范围
# gamma，即用于gamma矫正的一些参数，以矫正图像

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
ndwi = image.normalizedDifference(['B3', 'B5'])
# 计算两个波段之间归一化差异计算方法为两个波段之差除以两个波段之和
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
Map.setCenter(-122.1899, 37.5010, 9)
Map.addLayer(ndwi, ndwiViz, 'NDWI')
# palette，用于以彩色显示单波段的图像，由CSS样式的颜色字符串列表来建立色带显示图像

In [ ]:
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
Map.addLayer(ndwiMasked, ndwiViz, 'NDWI masked')
# updateMask,将不符合要求的点在图像上设置成不透明度为0

In [ ]:
imageRGB = image.visualize(**{'bands': ['B5',  'B4',  'B3'], 'max': 0.5})
ndwiRGB = ndwiMasked.visualize(**{
  'min': 0.5,
  'max': 1,
  'palette': ['00FFFF', '0000FF']
})
Map.addLayer(imageRGB, {}, "imageRGB")
Map.addLayer(ndwiRGB, {}, "ndwiRGB")
# visualize可视化图像的RGB和灰度值

In [ ]:
mosaic = ee.ImageCollection([imageRGB, ndwiRGB]).mosaic()
Map.addLayer(mosaic, {}, 'mosaic')
# 先用ImageCollection建立影像集，然后用.mosaic()按集合中的影像顺序绘制图层
# 实现一种镶嵌的效果

In [ ]:
roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)
Map.addLayer(mosaic.clip(roi))
# clip为裁剪图层